## Modelo "Otimizado"
Apenas um união dos códigos principais para ML.

In [1]:
import pandas as pd
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Já que durante os testes a coluna ['garage'] praticamente não afetou as predições, não a usaremos mais. 

In [6]:
df = pd.read_pickle('5 Apartamentos para modelagem.pkl')
df.reset_index(drop= True, inplace=True)
df.drop(columns=["price by m²", "garage"],inplace=True)
df.head()

,price,district,bedroom,bathroom,size
0,174000,Parnamirim,1,1,34.0
1,180000,Rosarinho,1,1,34.0
2,190000,Rosarinho,1,1,33.0
3,190000,Tamarineira,1,1,34.0
4,195000,Boa Viagem,1,1,30.0


In [7]:
dummies = pd.get_dummies(df.district)

data_frame = pd.concat([df,dummies], axis="columns")
data_frame.drop(columns="district",inplace=True)

X = data_frame.drop(columns="price")
y = data_frame.price

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

linear_model  = LinearRegression()

linear_model.fit(X_train,y_train)

linear_model.score(X_test,y_test)

0.8441350657700246

**Função que reberá as infromações de apartamentos e preverá seu valor. Irei utilizá-la na aplicação web.**

In [90]:
def predict_price(district, sqft, bedroom, bathroom):    
    loc_index = np.where(X.columns==district)[0][0]
   
    sample = np.zeros(len(X.columns))
    sample[0] = bedroom 
    sample[1] = bathroom
    sample[2] = sqft
       
    if loc_index >= 0:
        sample[loc_index] = 1

    return linear_model.predict([sample])[0]

Testando a função:

In [135]:
for test in range(5):
    data = df.sample()
    display(data)
    
    location = data.district.iloc[0]
    m2 = int(data['size'])
    bedroom = int(data.bedroom)
    bathroom = int(data.bathroom)
    
    print( "Predição", predict_price(location, m2, bedroom, bathroom) )

,price,district,bedroom,bathroom,size
1198,130000,Candeias,1,1,35.0


Predição 181342.0


,price,district,bedroom,bathroom,size
1885,380000,Madalena,3,2,66.0


Predição 376028.0


,price,district,bedroom,bathroom,size
3791,911000,Ilha do Retiro,2,1,135.0


Predição 721108.0


,price,district,bedroom,bathroom,size
2211,550000,Madalena,3,4,109.0


Predição 584086.5


,price,district,bedroom,bathroom,size
3687,800000,Espinheiro,3,2,140.0


Predição 677054.5


**Exportar o modelo para utilizarmos na aplicação.**

In [141]:
import pickle
with open('Recife_Imóveis_Model.pkl','wb') as file:
    pickle.dump(linear_model, file)

**Exportando nomes das localizações. Será útil mais tarde em nosso aplicativo.**

In [144]:
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("Recife_Imóveis_Model_columns.json","w") as f:
    f.write(json.dumps(columns))